## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [36]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']].copy()
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [37]:
# Create a list of at least 10 column names to use as X data
selected_features = [
    'Education',
    'Age',
    'DistanceFromHome',
    'JobSatisfaction',
    'OverTime',
    'StockOptionLevel',
    'WorkLifeBalance',
    'YearsAtCompany',
    'YearsSinceLastPromotion',
    'NumCompaniesWorked'
]

# Create X_df using your selected columns
X_df = attrition_df[selected_features].copy()

# Show the data types for X_df
X_df.dtypes


,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [38]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X = X_df  # your selected feature DataFrame
y_attrition  = y_df['Attrition']
y_department = y_df['Department']

# Split into train/test for both targets at once
X_train, X_test, \
y_attr_train, y_attr_test, \
y_dept_train, y_dept_test = train_test_split(
    X,
    y_attrition,
    y_department,
    test_size=0.2,
    random_state=42)

# Quick sanity‐check shapes
print(X_train.shape, X_test.shape)
print(y_attr_train.shape, y_attr_test.shape)
print(y_dept_train.shape, y_dept_test.shape)

print("Train OverTime counts:\n", X_train['OverTime'].value_counts(dropna=False))
print("\nTest OverTime counts:\n",  X_test ['OverTime'].value_counts(dropna=False))


(1176, 10) (294, 10)
(1176,) (294,)
(1176,) (294,)
Train OverTime counts:
 OverTime
No     837
Yes    339
Name: count, dtype: int64

Test OverTime counts:
 OverTime
No     217
Yes     77
Name: count, dtype: int64


In [39]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
# Convert OverTime to binary 0/1
X_train['OverTime'] = (X_train['OverTime'] == 'Yes').astype(int)
X_test['OverTime']  = (X_test['OverTime']  == 'Yes').astype(int)

# Print the updated counts
print("Train OverTime:\n", X_train['OverTime'].value_counts())
print("\nTest OverTime:\n",  X_test['OverTime'].value_counts())


Train OverTime:
 OverTime
0    837
1    339
Name: count, dtype: int64

Test OverTime:
 OverTime
0    217
1     77
Name: count, dtype: int64


In [40]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled  = scaler.transform(X_test)

In [42]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
dept_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
dept_encoder.fit(y_dept_train.values.reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_dept_train_ohe = dept_encoder.transform(y_dept_train.values.reshape(-1, 1))
y_dept_test_ohe  = dept_encoder.transform(y_dept_test.values.reshape(-1, 1))

print(y_dept_train_ohe[:5])


[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [43]:
# Create a OneHotEncoder for the Attrition column
attr_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attr_encoder.fit(y_attr_train.values.reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_attr_train_ohe = attr_encoder.transform(y_attr_train.values.reshape(-1, 1))
y_attr_test_ohe  = attr_encoder.transform(y_attr_test.values.reshape(-1, 1))

print(y_attr_train_ohe[:5])


[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


## Part 2: Create, Compile, and Train the Model

In [44]:
# Find the number of columns in the X training data.
n_features = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(X_train_scaled.shape[1],), name='input_features')

# Create at least two shared layers
shared1 = layers.Dense(64, activation='relu', name='shared1')(input_layer)
shared2 = layers.Dense(128, activation='relu', name='shared2')(shared1)

In [45]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu', name='department_hidden')(shared2)

# Create the output layer
department_output = layers.Dense(
    y_dept_train_ohe.shape[1],
    activation='softmax',
    name='department_output'
)(department_hidden)

In [46]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(
    32,
    activation='relu',
    name='attrition_hidden'
)(shared2)


# Create the output layer
attrition_output = layers.Dense(
    2,
    activation='softmax',
    name='attrition_output'
)(attrition_hidden)

In [48]:
# Create the model
model = Model(
    inputs=input_layer,
    outputs=[attrition_output, department_output],
    name='functional'
)

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'attrition_output': 'categorical_crossentropy',
        'department_output': 'categorical_crossentropy'
    },
    metrics={
        'attrition_output': 'accuracy',
        'department_output': 'accuracy'
    }
)


# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_features      │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared1 (Dense)     │ (None, 64)        │        704 │ input_features[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared2 (Dense)     │ (None, 128)       │      8,320 │ shared1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden    │ (None, 32)        │      4,128 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden   │ (None, 32)        │      4,128 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ department_hidde… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [49]:
# Train the model
model.fit(
    X_train_scaled,
    {
        'attrition_output': y_attr_train_ohe,
        'department_output': y_dept_train_ohe
    },
    validation_data=(
        X_test_scaled,
        {
            'attrition_output': y_attr_test_ohe,
            'department_output': y_dept_test_ohe
        }
    ),
    epochs=100,        # you can adjust
    batch_size=32,    # you can adjust
    validation_split=0.2
)

Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - attrition_output_accuracy: 0.6474 - attrition_output_loss: 0.5977 - department_output_accuracy: 0.5572 - department_output_loss: 0.9538 - loss: 1.5515 - val_attrition_output_accuracy: 0.8673 - val_attrition_output_loss: 0.3651 - val_department_output_accuracy: 0.6667 - val_department_output_loss: 0.7700 - val_loss: 1.1667
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - attrition_output_accuracy: 0.8241 - attrition_output_loss: 0.4439 - department_output_accuracy: 0.6569 - department_output_loss: 0.7592 - loss: 1.2032 - val_attrition_output_accuracy: 0.8605 - val_attrition_output_loss: 0.3577 - val_department_output_accuracy: 0.6667 - val_department_output_loss: 0.7709 - val_loss: 1.1586
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attrition_output_accuracy: 0.8332 - attrition_output_loss: 0.4129 - department_output_accuracy: 0.6433 - department_output_loss: 0.7687 - loss: 1.1816 - val_attrition_output_accuracy: 0.8435 -

In [51]:
# Evaluate the model with the testing data
results = model.evaluate(
    X_test_scaled,
    {
        'attrition_output': y_attr_test_ohe,
        'department_output': y_dept_test_ohe
    },
    verbose=2
)

print(model.metrics_names)
# ['loss',
#  'attrition_output_loss',
#  'department_output_loss',
#  'attrition_output_accuracy',
#  'department_output_accuracy']

# Unpack all five
loss_total, loss_attr, loss_dept, acc_attr, acc_dept = results

print(f"Total loss:           {loss_total:.4f}")
print(f"Attrition loss:       {loss_attr:.4f}")
print(f"Department loss:      {loss_dept:.4f}")
print(f"Attrition accuracy:   {acc_attr:.4f}")
print(f"Department accuracy:  {acc_dept:.4f}")

10/10 - 0s - 11ms/step - attrition_output_accuracy: 0.7925 - attrition_output_loss: 1.8359 - department_output_accuracy: 0.5136 - department_output_loss: 2.9098 - loss: 4.8387
['loss', 'compile_metrics', 'attrition_output_loss', 'department_output_loss']
Total loss:           4.8387
Attrition loss:       1.8359
Department loss:      2.9098
Attrition accuracy:   0.7925
Department accuracy:  0.5136


In [53]:
# Print the accuracy for both department and attrition
# Evaluate and print only the two accuracies
results = model.evaluate(
    X_test_scaled,
    {
        'attrition_output': y_attr_test_ohe,
        'department_output': y_dept_test_ohe
    },
    verbose=0
)

# Unpack the returned list
_, _, _, attrition_acc, department_acc = results

print(f"Attrition accuracy:  {attrition_acc:.4f}")
print(f"Department accuracy: {department_acc:.4f}")


Attrition accuracy:  0.7925
Department accuracy: 0.5136


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. In our case accuracy alone doesn’t tell the full story. Because only about 20 % of employees actually leave, a model that always predicts “stay” will score 80 % accuracy—even though it never identifies a single person at risk of quitting.
What really matters is how well we catch the leavers and correctly assign every department, not just the largest group. Tracking recall for attrition shows us the proportion of true leavers we successfully flag; precision tells us how many of our “will leave” warnings are correct; and the F₁‑score balances those two. For department prediction, a macro‑averaged F₁‑score treats each department equally, so small teams matter just as much as big ones.

2. For both the attrition and department heads I used a softmax activation. Softmax takes the raw scores from the final Dense layer and turns them into a probability distribution over the classes—in our case, two classes for attrition and three for department—so that the outputs always sum to 1. That makes it easy to interpret each head’s prediction as “this person has a 23 % chance to leave” or “this person belongs to R&D with 45 % probability,” and it pairs directly with the categorical_crossentropy loss we chose.

3. We could start by adding dropout or a small L2 penalty between layers and using early stopping. That will help prevent the network from memorizing noise and force it to learn patterns that actually generalize to new employees. We might also experiment with different layer sizes—say one wider hidden layer instead of two narrower ones—to see which architecture learns best without overfitting.

Because the “will leave” class is so rare, we should give it extra weight in the loss function or switch to a focal‑loss variant. That way the model pays more attention to the quitters instead of just coasting on the majority “stay” predictions. Running k‑fold cross‑validation instead of a single split will also give us a more reliable estimate of performance and reduce our chance of chasing a random train/test fluctuation.

Finally, it’s worth trying a non‑neural baseline like Random Forest or XGBoost. Those tree‑based methods often handle a mix of numeric and categorical data really well and can show us how much extra power the neural net is adding. We can also build simple interaction features (for example, YearsAtCompany × JobSatisfaction) or embeddings for any remaining categories to give the model richer inputs.